In [1]:
import numpy as np
import struct
import glob
import sys
import csv

In [2]:
filename = glob.glob(r'C:\Users\adamf\Documents\PhD\Diffraction\Files\*.DAT')[0]
print(filename)

C:\Users\adamf\Documents\PhD\Diffraction\Files\FILE_DS4293_001.DAT


In [3]:
with open(filename, 'rb') as f:
    data = f.read()
    print(type(data), len(data))

<class 'bytes'> 1811688


In [4]:
#for i in range(4):
#    print(i, struct.unpack('<4I', data[i:16+i]))
print(struct.unpack('>4I', data[8:16+8]))
print(struct.unpack('>I', data[8:8+4]))

(730712, 75905028, 6, 3273548713)
(730712,)


In [5]:
def get_freq(channel_number):
    """
    Takes a channel number and returns the corresponding frequency in MHz.
    
    Args:
    channel_number (int): The channel number.
    
    Returns:
    float: The corresponding frequency in MHz.
    """
    # Mapping of channel numbers to frequencies
    channel_to_frequency = {
        1: 0.45,
        2: 0.70,
        3: 0.90,
        4: 1.31,
        5: 2.20,
        6: 3.93,
        7: 4.70,
        8: 6.55,
        9: 9.18
    }
    
    # Get the frequency corresponding to the channel number
    return channel_to_frequency.get(channel_number, "Invalid channel number")

In [6]:
class Rae2File:
    def __init__(self, filename):
        with open(filename, 'rb') as f:
            self.raw_data = f.read()  # skip first junk bytes
        self.offset = 4

    def read_date(self):    
        i = self.offset
        self.offset += 4
        date = struct.unpack('>I', self.raw_data[i:i+4])
        self.offset += 4
        return date
        
    def read_block(self):
        i = self.offset
        assert len(self.raw_data[i:]) > 4 * 28
        ms, fq_band = struct.unpack('>II', self.raw_data[i:i+8])
        i += 8
        rdata = struct.unpack('>26f', self.raw_data[i:i+4*26])
        self.offset += 4 * 28
        return ms, fq_band, np.array(rdata)

    def read_logical_record(self, cnt=8):
        #self.offset = self.block_start
        date = self.read_date()
        blocks = []
        for i in range(cnt):
            blocks.append(self.read_block())
        temp = self.read_temp()
        return date, blocks, temp
        
    def read_temp(self):
        temp = struct.unpack('>I', self.raw_data[self.offset:self.offset+4])
        self.offset += 4
        return temp

    def read_physical_record(self, cnt=34):
        data = []
        for i in range(cnt):
            data.append(self.read_logical_record())
        self.offset += 4
        return data

    def read_file(self):
        data = []
        while True:
            try:
                data.append(self.read_physical_record())
            except(AssertionError):
                break
        return data

In [7]:
f = Rae2File(filename)

In [8]:
data = f.read_file()
print(len(data))
#hex(73)
#hex(730615)

58


In [10]:
data[0]

[((59506688,),
  [(75905028,
    6,
    array([-158.38929749,  275.58404541, -232.23750305,    2.57009149,
              7.18172932,    3.66753793,   14.1392231 ,  -40.24690247,
              0.        ,    0.        ,    0.        ,    0.        ,
              0.        ,    0.        ,    0.        ,    0.        ,
              0.        ,    0.        ,    0.        ,    0.        ,
              0.        ,    0.        ,    0.        ,    0.        ,
              0.        ,    0.        ])),
   (76043257,
    6,
    array([-169.50268555,  273.12188721, -229.98345947,    2.56878853,
              7.1821723 ,    3.66750622,   14.43065453,  -40.11903   ,
              0.        ,    0.        ,    0.        ,    0.        ,
              0.        ,    0.        ,    0.        ,    0.        ,
              0.        ,    0.        ,    0.        ,    0.        ,
              0.        ,    0.        ,    0.        ,    0.        ,
              0.        ,    0.        ])),
   

In [101]:
data[1][0][0]

(730712,)

In [95]:
def dataToCSV(data,filename):
    with open(filename,'w',newline = '') as csvfile:
        swriter = csv.writer(csvfile)
        swriter.writerow(['Date (YYMMDD)','Milliseconds of Day', 'Freq(MHz)','Spacecraft X (KM)', 'Spacecraft Y(KM)',
                             'Spacecraft Z (KM)','Unit Vector X','Unit Vector Y','Unit Vector Z','RA','Dec']+['Coarse RV-1']*7
                           +['Fine RV-1']*2+['Coarse RV-2']*7+['Fine RV-2']*2+['Temperature'])
        for i in range(len(data)):
            for j in range(len(data[i])):
                for k in range(1,len(data[i][j][1])):
                    date = int(*data[i][j][0])
                    freq = get_freq(int(data[i][j][1][k][0]))
                    mS = int(data[i][j][1][k][1])
                    data = data[i][j][1][k][2]
                    #temp = type(data[i][j][2])
                    #swriter.writerow([date,freq,mS,data,temp])
                    swriter.writerow([date,freq,mS,data])
    return -1

In [96]:
dataToCSV(data,'test.csv')

59506688


IndexError: invalid index to scalar variable.

In [168]:
f.offset, len(f.raw_data)

(1811700, 1811688)

In [39]:
for i in range

'hd' is not recognized as an internal or external command,
operable program or batch file.
